In [9]:
# %cd data
# !pwd
# !gunzip GoogleNews-vectors-negative300.bin
# %cd ..

# print(type(model))
# print(type('ds'))
# print(type((1,2)))

# print(type(df))
# test = word_tokenize('The PlayStation 4 system opens the door to an incredible journey through immersive new gaming worlds and a deeply connected gaming community. Step into living, breathing worlds where you are hero of your epic journey. Explore gritty urban environments, vast galactic landscapes, and fantastic historical settings brought to life on an epic scale, without limits. With an astounding launch lineup and over 180 games in development the PS4 system offers more top-tier blockbusters and inventive indie hits than any other next-gen console. The PS4 system is developer inspired, gamer focused. The PS4 system learns how you play and intuitively curates the content you use most often. Fire it up, and your PS4 system points the way to new, amazing experiences you can jump into alone or with friends. Create your own legend using a sophisticated, intuitive network built for gamers. Broadcast your gameplay live and direct to the world, complete with your commentary. Or immortalize your most epic moments and share at the press of a button. Access the best in music, movies, sports and television. PS4 system doesn t require a membership fee to access your digital entertainment subscriptions. You get the full spectrum of entertainment that matters to you on the PS4 system. PlayStation 4: The Best Place to Play The PlayStation 4 system provides dynamic, connected gaming, powerful graphics and speed, intelligent personalization, deeply integrated social capabilities, and innovative second-screen features. Combining unparalleled content, immersive gaming experiences, all of your favorite digital entertainment apps, and PlayStation exclusives, the PS4 system focuses on the gamers.Gamer Focused, Developer InspiredThe PS4 system focuses on the gamer, ensuring that the very best games and the most immersive experiences are possible on the platform.<br>Read more about the PS4 on ebay guides.</br>')
# print(type(stop_words))
# query = preprocess('The PlayStation 4 system opens the door to an incredible journey through immersive new gaming worlds and a deeply connected gaming community. Step into living, breathing worlds where you are hero of your epic journey. Explore gritty urban environments, vast galactic landscapes, and fantastic historical settings brought to life on an epic scale, without limits. With an astounding launch lineup and over 180 games in development the PS4 system offers more top-tier blockbusters and inventive indie hits than any other next-gen console. The PS4 system is developer inspired, gamer focused. The PS4 system learns how you play and intuitively curates the content you use most often. Fire it up, and your PS4 system points the way to new, amazing experiences you can jump into alone or with friends. Create your own legend using a sophisticated, intuitive network built for gamers. Broadcast your gameplay live and direct to the world, complete with your commentary. Or immortalize your most epic moments and share at the press of a button. Access the best in music, movies, sports and television. PS4 system doesn t require a membership fee to access your digital entertainment subscriptions. You get the full spectrum of entertainment that matters to you on the PS4 system. PlayStation 4: The Best Place to Play The PlayStation 4 system provides dynamic, connected gaming, powerful graphics and speed, intelligent personalization, deeply integrated social capabilities, and innovative second-screen features. Combining unparalleled content, immersive gaming experiences, all of your favorite digital entertainment apps, and PlayStation exclusives, the PS4 system focuses on the gamers.Gamer Focused, Developer InspiredThe PS4 system focuses on the gamer, ensuring that the very best games and the most immersive experiences are possible on the platform.<br>Read more about the PS4 on ebay guides.</br>')
# print(type(query))
# print(query)

# !pwd
# !ls "data/"
# !wget -c "http://dumps.wikimedia.org/other/articlefeedback/aa_combined-20110321.csv.gz" -P "data/"
# !ls
# !gunzip "data/aa_combined-20110321.csv.gz"
# !ls "data/"

# url_to_word_embedding_zip = "http://dumps.wikimedia.org/other/articlefeedback/aa_combined-20110321.csv.gz"
# os.path.splitext(url_to_word_embedding_zip)[1][1:]

# from gensim.models.keyedvectors import KeyedVectors
# model_path = './data/GoogleNews-vectors-negative300.bin'
# model = KeyedVectors.load_word2vec_format(model_path, binary=True)

# import gensim
# print(type(model))

# isinstance(model, gensim.models.keyedvectors.Word2VecKeyedVectors)

# import pandas as pd
# a = pd.DataFrame({'a':[1,2], 'b':[3,4]})
# def letgo(df):
#     df['b'] = [5,6]
#     return df
# letgo(a)
# print(a)
# a.drop('b',axis=1)
# print(a)

# #Create pandas dataframe on dataset
# import pandas as pd
# #https://data.world/crowdflower/ecommerce-search-relevance
# df = pd.read_csv('https://query.data.world/s/iccclhgzbyedtv54p5lresglnun3qz', encoding = "ISO-8859-1")
# df = df.iloc[0:10]
# print(df)
# print(type(df))

import pandas as pd
#https://data.world/crowdflower/ecommerce-search-relevance
df = pd.read_csv('https://query.data.world/s/iccclhgzbyedtv54p5lresglnun3qz', encoding = "ISO-8859-1")
df = df.iloc[0:10]
type(df["product_description"][2])
df["product_description"][2] == df["product_description"][2]
print(type(df["product_description"][2]))


<class 'str'>


In [0]:
#INPUTS

#repoPath should start with the path 
#'/content/gdrive/My Drive/' and go to the folder where the git repo is, and should end with the name of the repo:
repoPath = '/content/gdrive/My Drive/TestingGitCloneIntoGoogleDrive/textSimilarityUsingWordMoversDistance'
userEmail = 'sanmesh1@gmail.com'
userName = 'sanmesh1'

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [12]:
! pip install ipdb -q
import ipdb 

#place the line of code below from whichever line you want to start debugging
#type n for step over and s for step into
# ipdb.set_trace()

In [0]:
import os
os.chdir(repoPath)
#%cd gdrive/My Drive/project_folder/TextSimilarityUsingWord2Vec

In [0]:
from time import time
start_nb = time()

In [15]:
# Import and download stopwords from NLTK.
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.

# Remove stopwords.
stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
start = time()
import os
from gensim.models import Word2Vec

%cd data/
!ls
import os.path
from os import path
if path.exists("GoogleNews-vectors-negative300.bin") == False:
  !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
  !gunzip GoogleNews-vectors-negative300.bin
else:
  print("datasetAlreadyExists")
%cd ..

from gensim.models.keyedvectors import KeyedVectors
model_path = './data/GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(model_path, binary=True)

print('Cell took %.2f seconds to run.' % (time() - start))

/content/gdrive/My Drive/TestingGitCloneIntoGoogleDrive/textSimilarityUsingWordMoversDistance/data
aa_combined-20110321.csv  GoogleNews-vectors-negative300.bin
datasetAlreadyExists
/content/gdrive/My Drive/TestingGitCloneIntoGoogleDrive/textSimilarityUsingWordMoversDistance


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Cell took 82.71 seconds to run.


In [17]:
# Pre-processing a document.

from nltk import word_tokenize
download('punkt')  # Download data for tokenizer.

def preprocess(doc):
    doc = doc.lower()  # Lower the text.
    doc = word_tokenize(doc)  # Split into words.
    doc = [w for w in doc if not w in stop_words]  # Remove stopwords.
    doc = [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    return doc

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
#Create pandas dataframe on dataset
import pandas as pd
#https://data.world/crowdflower/ecommerce-search-relevance
df = pd.read_csv('https://query.data.world/s/iccclhgzbyedtv54p5lresglnun3qz', encoding = "ISO-8859-1")

In [0]:
#create source document and target document we will be comparing scores for
numDataPoints = 3000
sourceIndex = 2584
sourceTitle = df['product_title'][sourceIndex]
source_doc = df['product_description'][sourceIndex] #2584

targetTitles = df['product_title'][:numDataPoints].tolist()
target_docs = df['product_description'][:numDataPoints].tolist()
targetQueries = df['query'][:numDataPoints].tolist()
targetRelevance = df['relevance'][:numDataPoints].tolist()

In [0]:
#using WmdSimilarity

wmd_corpus = []
documents = []
documentTitles = []
documentQueries = []
documentRelevance = []

for i in range(len(target_docs)):
    if target_docs[i] == target_docs[i]: #this is to check if there is an empty string
      # ipdb.set_trace()
      target_docs_preproc = preprocess(target_docs[i])
      if target_docs_preproc != []:
        wmd_corpus.append(target_docs_preproc)
      else:
        wmd_corpus.append([])
    else:
      wmd_corpus.append([])
    documentTitles.append(targetTitles[i])
    documents.append(target_docs[i])
    documentQueries.append(targetQueries[i])
    documentRelevance.append(targetRelevance[i])
# Initialize WmdSimilarity.
from gensim.similarities import WmdSimilarity
num_best = 10
instance = WmdSimilarity(wmd_corpus, model, num_best=10)



In [0]:
start = time()

query = preprocess(source_doc)

sims = instance[query]  # A query is simply a "look-up" in the similarity class.

In [24]:
print(query)
type(sims[0])
sims

['go', 'green', 'aaa', 'alkaline', 'batteries', 'independently', 'tested', 'verify', 'performance', 'like', 'expensive', 'brands', 'environmentally', 'friendly', 'contains', 'lead', 'cadmium', 'mercury', 'made', 'recycled', 'materials']


[(2584, 1.0),
 (2593, 0.4990286299039699),
 (2597, 0.4985625604198936),
 (2595, 0.4969591472144237),
 (2596, 0.4864639593072192),
 (2600, 0.4859263225295364),
 (2594, 0.48531136112789613),
 (2598, 0.48404949014983617),
 (2601, 0.48404949014983617),
 (2599, 0.4839526592578422)]

In [57]:
# Print the query and the retrieved documents, together with their similarities.
print('Query:')
print(sourceTitle)
for i in range(num_best):
    print()
    print('sim in description = %.4f' % sims[i][1])
    print(documentTitles[sims[i][0]])
    simInTitles = model.wmdistance(sourceTitle, documentTitles[sims[i][0]])
    print("loss in title similarity = ", simInTitles)
    print("item query = ", documentQueries[sims[i][0]])

'''
if index_of_query_product "relevance" field < score_cutoff: #we dont want to use query items in which the similarity to the search query was low.
  return accuracy = None
else: 
  for every index in similar_document_tuple_of_indexes_and_similarity_scores:
    number_of_correct_similar_products += (index_of_query_product "query" field == similar_index "query" field)*(similar_index "relevance" field >= score_cutoff)
accuracy = number_of_correct_similar_products/num_similar_queries
'''
score_cutoff =2
number_of_correct_similar_products = 0
if df['relevance'][sourceIndex] < score_cutoff: #we dont want to use query items in which the similarity to the search query was low.
  accuracy = None
else: 
  for i in range(num_best):
    number_of_correct_similar_products += (df['query'][sourceIndex] == documentQueries[sims[i][0]]) * (documentRelevance[sourceIndex] >= score_cutoff)
accuracy = number_of_correct_similar_products/num_best

print("accuracy = ", accuracy)

Query:
Go Green AAA Alkaline Battery 8-pack

sim in description = 1.0000
Go Green AAA Alkaline Battery 8-pack
loss in title similarity =  0.0
item query =  Rechargable batteries

sim in description = 0.4990
Duracell Procell Alkaline AAA Batteries (Case of 24)
loss in title similarity =  0.3564070574522147
item query =  Rechargable batteries

sim in description = 0.4986
Duracell Coppertop Alkaline AAA Batteries (Case of 24)
loss in title similarity =  0.32415265200084475
item query =  Rechargable batteries

sim in description = 0.4970
Energizer Industrial Alkaline AAA Batteries (Case of 24)
loss in title similarity =  0.3912427266011484
item query =  Rechargable batteries

sim in description = 0.4948
SmartyKat SweetGreens Cat Grass Kit
loss in title similarity =  0.586983153963638
item query =  cat grass

sim in description = 0.4865
Innovera Alkaline AA Batteries (Pack of 8)
loss in title similarity =  0.2779739110684397
item query =  Rechargable batteries

sim in description = 0.4859
R

In [0]:
def calculatePercentageOfSimilarItemsInSameQueryCategory(instance, df, num_best, numSamples):
  numRows = df.shape[0]
  numMatchingCategory = 0
  numNonMatchingCategory = 0
  # for i in range(numRows):  
  for i in range(numSamples):  
    print(i)
    source_doc_query = df['query'][i]
    if source_doc_query == source_doc_query:
      query = preprocess(source_doc_query)
      sims = instance[query]  # A query is simply a "look-up" in the similarity class.
      for i in range(num_best):
        if documentQueries[sims[i][0]] == source_doc_query:
          numMatchingCategory += 1
        else:
          numNonMatchingCategory
  print("numMatchingCategory = ", numMatchingCategory)
  print("numNonMatchingCategory = ", numNonMatchingCategory)
  return numMatchingCategory, numNonMatchingCategory



In [0]:
numMatchingCategory, numNonMatchingCategory =calculatePercentageOfSimilarItemsInSameQueryCategory(instance, df, num_best, 10)

0
1
2
3
4
5
6
7
8
9
numMatchingCategory =  20
numNonMatchingCategory =  0
